In [1]:
f = open(r'/home/cloudera/Desktop/educational/ADA1/Programming Assignment 5/dijkstraData.txt')
G = {}

In [2]:
# Lay out the adjacency list representation of the graph as a python dict
for line in f:
	s = line.split()[0]
	v_list = line.split()[1:]
	G[s] = v_list
f.close()

In [3]:
# create the set V that stores all vertices
V = set(G.keys())
# create the dict that stores the shortest paths to each node from the node '1'
A = {}
for k in G.keys():
	A[k] = 1000000

In [4]:
#print G
#print A
#print V

We would like to represent the Graph as a list of edges (with the corresponding edge-lengths) in order to facilitate Dijkstra's algorithm. We can do this using a map-only hadoop job. Create a file named **mapper.py** with the below code:

    import sys
    for line in sys.stdin:
        key = line.split()[0]
        value = line.split()[1:]
        for v in value:
            print '%s,%s' %(key, v)
            
Then run the below commands:

    [cloudera@localhost Programming Assignment 5]$ hadoop fs -put dijkstraData.txt
    [cloudera@localhost Programming Assignment 5]$ hadoop jar $STREAMING -input dijkstraData.txt -output Prog_ass_5 -mapper "python mapper.py" -file mapper.py

    packageJobJar: [mapper.py, /tmp/hadoop-cloudera/hadoop-unjar2341889022427608157/] [] /tmp/streamjob5268864320777423389.jar tmpDir=null
    15/08/08 19:47:59 WARN mapred.JobClient: Use GenericOptionsParser for parsing the arguments. Applications should implement Tool for the same.
    15/08/08 19:47:59 INFO mapred.FileInputFormat: Total input paths to process : 1
    15/08/08 19:48:00 INFO streaming.StreamJob: getLocalDirs(): [/tmp/hadoop-cloudera/mapred/local]
    15/08/08 19:48:00 INFO streaming.StreamJob: Running job: job_201508081707_0001
    15/08/08 19:48:00 INFO streaming.StreamJob: To kill this job, run:
    15/08/08 19:48:00 INFO streaming.StreamJob: UNDEF/bin/hadoop job  -Dmapred.job.tracker=localhost.localdomain:8021 -kill job_201508081707_0001
    15/08/08 19:48:00 INFO streaming.StreamJob: Tracking URL: http://0.0.0.0:50030/jobdetails.jsp?jobid=job_201508081707_0001
    15/08/08 19:48:01 INFO streaming.StreamJob:  map 0%  reduce 0%
    15/08/08 19:48:13 INFO streaming.StreamJob:  map 100%  reduce 0%
    15/08/08 19:48:18 INFO streaming.StreamJob:  map 100%  reduce 100%
    15/08/08 19:48:21 INFO streaming.StreamJob: Job complete: job_201508081707_0001
    15/08/08 19:48:21 INFO streaming.StreamJob: Output: Prog_ass_5

The job was successful. Now we need to look into the output directory to make sure that the output is as expected.

    [cloudera@localhost Programming Assignment 5]$ hadoop fs -ls Prog_ass_5
    Found 3 items
    -rw-r--r--   3 cloudera cloudera          0 2015-08-08 19:48 Prog_ass_5/_SUCCESS
    drwxr-xr-x   - cloudera cloudera          0 2015-08-08 19:48 Prog_ass_5/_logs
    -rw-r--r--   3 cloudera cloudera         36 2015-08-08 19:48 Prog_ass_5/part-00000

Just to make sure that the output file is in the format we expect, lets run the below command:
    [cloudera@localhost Programming Assignment 5]$ hadoop fs -cat Prog_ass_5/part*
    1,2,3	
    1,3,3	
    2,3,1	
    2,4,2	
    3,4,50

Everything looks good. So, lets get the output file from the HDFS onto the local file system.

    [cloudera@localhost Programming Assignment 5]$ hadoop fs -cat Prog_ass_5/part* > edgelist.txt
    [cloudera@localhost Programming Assignment 5]$ cat edgelist.txt
    1,2,3	
    1,3,3	
    2,3,1	
    2,4,2	
    3,4,50	

In [5]:
# Read in the edge list
E = {}
i = 0
f = open(r'/home/cloudera/Desktop/educational/ADA1/Programming Assignment 5/edgelist.txt')
for line in f:
    data = line.strip().split(',')
    E[i] = {'Node1':data[0], 'Node2':data[1], 'Length':data[2]}
    i+=1

In [6]:
# Start Dijkstra's algorithm
# Initialize X and A
X = set(['1'])
A['1'] = 0

In [7]:
# The main while loop
while X<>V:
    # for each edge that has tail in X and head in V
    l = []
    for e in E.keys():
        if (E[e]['Node1'] in X) and (E[e]['Node2'] not in X):
            l.append((E[e]['Node2'], A[E[e]['Node1']] + int(E[e]['Length'])))

    N2, A2 = min(l, key = lambda x:x[1])
    X.add(N2)
    A[N2] = A2

In [18]:
#print X
#print V
print A['188'], A['197']

2505 3068


The shortest path from node '0' to node '188' is 2505. The shortest path from node '0' to node '197' is 3068.